In [11]:
# Importamos las librerías necesarias para convertir el pdf a texto
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

# Definimos la función que convierte el pdf a texto
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [12]:
# Guardamos el texto en un nuevo archivo
with open('Madrid_Parques.txt', 'w', encoding='utf-8') as f:
    text = convert_pdf_to_txt('Madrid_Parques.pdf')
    f.write(text)

In [1]:
# Leemos el texto por líneas
with open('Madrid_Parques.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

In [2]:
# Extraemos los nombres de los centros
centros = []
for i, line in enumerate(lines):
    if line.startswith('MADRID'):
        for j in range(i+2, len(lines)):
            if lines[j] == '\n':
                break
            else:
                centros.append(lines[j])

# Limpiamos y juntamos
centros = [centro.replace('\n', '').replace('  ', ' ') for centro in centros]
centros = [centros[0] + ' ' + centros[1] + ' ' + centros[2],
            centros[3] + ' ' + centros[4][:-1] + centros[5],
            centros[6] + ' ' + centros[7],
            centros[8] + ' ' + centros[9][:-1] + centros[10]]
centros = [(centro.split('.')[0], centro.split('.')[1][1:]) for centro in centros]

In [3]:
centros

[('Parque Natural de Peñalara',
  'Centro de Educación Ambiental Puente del Perdón'),
 ('Parque Regional de la Cuenca Alta del Manzanares',
  'Centro de Educación Ambiental Manzanares'),
 ('Reserva de la Biosfera de la Sierra del Rincón', 'Hayedo de Montejo'),
 ('Reserva de la Biosfera de la Sierra del Rincón',
  'Senda del Agua y Laguna del Salmoral')]

In [35]:
import pandas as pd

def get_general_data(centros, lines):
    # Obtenemos datos de cada centro
    data1 = {'Ubicacion':[], 'Centro':[], 'Direccion':[], 'Telefono':[], 'email':[],'Informacion':[]}

    clean_lines = [line.replace('\n', '') for line in lines]

    for centro in centros:

        data1['Ubicacion'].append(centro[0])
        data1['Centro'].append(centro[1])

        for i, line in enumerate(clean_lines):
            if line.startswith(centro[1]) and clean_lines[i-1].startswith(centro[0][:10]):
                for j in range(i+2, len(clean_lines)):
                    if len(data1['Informacion']) == len(data1['Centro']):
                        break
                    elif clean_lines[j].startswith('Información General'):
                        texto = ''
                        for k in range(j+2, len(clean_lines)):
                            if clean_lines[k] == '':
                                data1['Informacion'].append(texto)
                                break
                            else:
                                texto += ' ' + clean_lines[k]
                    if j == i + 2:
                        # Distintas opciones en función del tipo de dirección
                        if clean_lines[j+1].startswith('Tel'):
                            data1['Direccion'].append(clean_lines[j])
                            data1['Telefono'].append(clean_lines[j+1])
                            data1['email'].append(clean_lines[j+2])

                        elif clean_lines[j+2].startswith('Tel'):
                            data1['Direccion'].append(clean_lines[j] + ' ' + clean_lines[j+1])
                            data1['Telefono'].append(clean_lines[j+2])
                            data1['email'].append(clean_lines[j+3])

                        else:
                            data1['Direccion'].append(clean_lines[j] + ' ' + clean_lines[j+2])
                            data1['Telefono'].append(clean_lines[j+3])
                            data1['email'].append(clean_lines[j+4])

    centros_df = pd.DataFrame(data1, index=range(len(data1['Centro'])))

    return centros_df

In [33]:
data1['Informacion']

[]

In [36]:
for key, val in data1.items():
    print(key, len(val))

Ubicacion 4
Centro 4
Direccion 4
Telefono 4
email 4
Informacion 4


In [27]:
for centro in centros:
    print(centro[1])
    print(centro[0][:10])

    for i, line in enumerate(clean_lines):
        if line.startswith(centro[1]) and clean_lines[i-1].startswith(centro[0][:10]):
            print('Se cumple la condición')
            print(line)
    print('-----------------------------')

Centro de Educación Ambiental Puente del Perdón
Parque Nat
Se cumple la condición
Centro de Educación Ambiental Puente del Perdón
-----------------------------
Centro de Educación Ambiental Manzanares
Parque Reg
Se cumple la condición
Centro de Educación Ambiental Manzanares 
-----------------------------
Hayedo de Montejo
Reserva de
Se cumple la condición
Hayedo de Montejo - “Senda del Río”
-----------------------------
Senda del Agua y Laguna del Salmoral
Reserva de
Se cumple la condición
Senda del Agua y Laguna del Salmoral
-----------------------------


In [37]:
centros_df

,Ubicacion,Centro,Direccion,Telefono,email,Informacion
0,Parque Natural de Peñalara,Centro de Educación Ambiental Puente del Perdón,Carretera M-604 Km. 28 28740 Rascafría (Madrid),Teléfono / Fax: 918 691 757,Email: redcentrospuentedp@yahoo.es,El “Centro de Educación Ambiental Puente del ...
1,Parque Regional de la Cuenca Alta del Manzanares,Centro de Educación Ambiental Manzanares,Camino de la Pedriza s/n 28410 Manzanares el R...,Teléfono / Fax: 918 539 978,Email: redcentros.manzanares@madrid.org,El Centro dispone de un edificio principal co...
2,Reserva de la Biosfera de la Sierra del Rincón,Hayedo de Montejo,Desde Montejo de la Sierra por la M- 137 hacia...,Teléfono: 91 869 70 58 / 91 869 72 17,Email: sierradelrincon@fida.es,El “Hayedo de Montejo” constituye uno de los ...
3,Reserva de la Biosfera de la Sierra del Rincón,Senda del Agua y Laguna del Salmoral,28191 Prádena del Rincón (Madrid),Teléfono: 91 869 70 58 / 91 869 72 17,Email: sierradelrincon@fida.es,La “Senda del Agua” une el municipio de Práde...


In [38]:
centros_df.to_csv('centros_madrid.csv')

In [47]:
def get_access_data(centros, lines):
    # Obtenemos accesibilidad de cada centro
    data2 = {'Centro':[]}

    clean_lines = [line.replace('\n', '') for line in lines]

    excepciones = ['Espacios y Actividades en la Naturaleza Accesibles para Todas las Personas',
                'Centros de interpretación / Casas del Parque y Aulas de la Naturaleza',
                'Ubicación',
                'Características destacables',
                'Zona de Picnic',
                '(cid:2) Dispone de paneles informati-',
                'trastado: Sí',
                'rrelieve: No',
                '(cid:2) Iconos homologados: no hay',
                'Señalización',
                'Sendero Guiado / Autoguiado',
                'pavimento',
                'Arboreto']

    found_cols = [] # Columnas que se añaden por centro

    # Extracción de datos
    for centro in centros:

        data2['Centro'].append(centro[1])
        print(len(data2['Centro']))

        for i, line in enumerate(clean_lines):
            if line.startswith(centro[1]) and clean_lines[i-1].startswith(centro[0][:10]):
                for j in range(i+2, len(clean_lines)):
                    if clean_lines[j] == 'Accesibilidad Visual':
                        break
                    elif clean_lines[j] == 'Accesibilidad Física':
                        for k in range(j+1, len(clean_lines)-1):
                            # Fin de los datos del centro
                            if clean_lines[k] == 'Accesibilidad Visual':
                                print(found_cols)
                                # Se mira el número de columnas y se añade 'NP' en las que existen pero no se han encontrado
                                for key in data2.keys():
                                    if key not in found_cols and key != 'Centro':
                                        data2[key].append('NP')
                                found_cols = [] # Se reinicia la cuenta
                                break
                            # Columna encontrada
                            elif clean_lines[k] != '' and clean_lines[k+1] == '' and clean_lines[k-1] == '' and clean_lines[k].replace(' ', '').isalpha() and '' not in clean_lines[k] and clean_lines[k] not in excepciones:
                                found_cols.append(clean_lines[k])
                                texto = ''
                                # Concatenando líneas de la respectiva columna
                                for n in range(k+2, len(clean_lines)):
                                    #print(len(data2['Centro']))
                                    if clean_lines[n] == '':
                                        if clean_lines[k] in data2.keys():
                                            data2[clean_lines[k]].append(texto.replace('- ', ''))
                                            break
                                        elif len(data2['Centro']) == 1:
                                            data2[clean_lines[k]] = [texto.replace('- ', '')]
                                            break
                                        else:
                                            data2[clean_lines[k]] = ['NP' for v in range(len(data2['Centro'])-1)] + [texto.replace('- ', '')]
                                            break
                                    # Nos saltamos los cambios de página.
                                    elif clean_lines[n] == '' and len(clean_lines[n+1]) == 1 or len(clean_lines[n]) == 1 and len(clean_lines[n+1]) == 1:
                                        continue
                                    else:
                                        texto += clean_lines[n] + ' '


    access_df = pd.DataFrame(data2, index=range(len(data2['Centro'])))

    return access_df

1
Entrando con Centro de Educación Ambiental Puente del Perdón
['Acceso ', 'Recepción', 'Aseo adaptado', 'Inodoro', 'Lavabo y accesorios del aseo', 'Senderos', 'Mobiliario y zonas de descanso']
2
Entrando con Centro de Educación Ambiental Manzanares
['Acceso ', 'Recepción', 'Salas de exposiciones', 'Sala de audiovisuales', 'Sala de talleres infantiles', 'Biblioteca', 'Aseo adaptado', 'Inodoro', 'Lavabo y accesorios del aseo']
3
Entrando con Hayedo de Montejo
['Itinerario de la Senda', 'Seguridad', 'Mobiliario y zonas de descanso', 'Aseo adaptado']
4
Entrando con Senda del Agua y Laguna del Salmoral
['Seguridad', 'Mobiliario y zonas de descanso', 'Observatorio', 'Aseo adaptado']


In [48]:
access_df

,Centro,Acceso,Recepción,Aseo adaptado,Inodoro,Lavabo y accesorios del aseo,Senderos,Mobiliario y zonas de descanso,Salas de exposiciones,Sala de audiovisuales,Sala de talleres infantiles,Biblioteca,Itinerario de la Senda,Seguridad,Observatorio
0,Centro de Educación Ambiental Puente del Perdón,Para acceder desde el aparcamiento hasta el ed...,Se accede desde las áreas temáticas al aire li...,"Hay una cabina de aseo adaptada, independiente...",El inodoro tiene un espacio libre de acercamie...,El lavabo permite la aproximación frontal de u...,Las sendas que recorren el “Área de Recursos A...,Dispone de una zona de descanso al aire libre ...,NP,NP,NP,NP,NP,NP,NP
1,Centro de Educación Ambiental Manzanares,Desde el aparcamiento hasta el edificio princi...,El itinerario desde la puerta de acceso hasta ...,Hay una cabina de aseo adaptada para ambos sex...,El inodoro tiene un espacio libre de acercamie...,El lavabo permite la aproximación frontal de u...,NP,NP,La sala de exposición está situada en la plant...,Se accede directamente desde la recepción sin ...,La puerta de entrada tiene dos hojas de madera...,Está situada en la primera planta del edificio...,NP,NP,NP
2,Hayedo de Montejo,NP,NP,La “Senda del Río” no tiene aseos.,NP,NP,NP,No dispone de mobiliario o zona de descanso co...,NP,NP,NP,NP,"Tras algo más de 1 km de recorrido, con un pav...",La senda está delimitada de manera puntual por...,NP
3,Senda del Agua y Laguna del Salmoral,NP,NP,La “Senda del agua” no tiene aseos.,NP,NP,NP,Dispone de alguna zona de descanso accesible p...,NP,NP,NP,NP,NP,La laguna está delimitada por bordillos latera...,"El observatorio es una caseta de madera, donde..."


In [49]:
access_df.to_csv('parques_madrid.csv')

In [45]:
for key, val in data2.items():
    try:
        print(key, len(val))
    except:
        print(key, val)

Centro 4
Acceso  4
Recepción 4
Aseo adaptado 4
Inodoro 4
Lavabo y accesorios del aseo 4
Senderos 4
Mobiliario y zonas de descanso 4
Salas de exposiciones 5
Sala de audiovisuales 5
Sala de talleres infantiles 5
Biblioteca 5
Itinerario de la Senda 5
Seguridad 5
Observatorio 5


In [46]:
data2['Observatorio']

['NP',
 'NP',
 'NP',
 'NP',
 'El observatorio es una caseta de madera, donde el suelo del itinerario está enrasado con el del observatorio. Se entra por un hueco libre de paso de 80 cm de ancho. Las franjas de observación están a 2 alturas, la alta es de 1,58 m y la baja de 95 cm. La franja baja no dispone de un espacio de aproximación frontal para usuarios de silla de ruedas. Dentro del observatorio hay un espacio libre de giro de más de 1,50 m de diámetro. Hay bancos de madera fijos situados frente a las franjas de observación.  ']

In [133]:
clean_lines[1143]

'Hayedo de Montejo - “Senda del Río”'

In [19]:
clean_lines[1396]

'Teléfono: 91 869 70 58 / 91 869 72 17'

In [140]:
centros[-1][1]

'Senda del Agua y Lagna del Salmoral'

In [1]:
import pandas as pd

c1 = pd.read_csv('centros_madrid.csv')
p1 = pd.read_csv('parques_madrid.csv')

In [2]:
c1

,Unnamed: 0,Ubicacion,Centro,Direccion,Telefono,email,Informacion
0,0,Parque Natural de Peñalara,Centro de Educación Ambiental Puente del Perdón,Carretera M-604 Km. 28 28740 Rascafría (Madrid),Teléfono / Fax: 918 691 757,Email: redcentrospuentedp@yahoo.es,El “Centro de Educación Ambiental Puente del ...
1,1,Parque Regional de la Cuenca Alta del Manzanares,Centro de Educación Ambiental Manzanares,Camino de la Pedriza s/n 28410 Manzanares el R...,Teléfono / Fax: 918 539 978,Email: redcentros.manzanares@madrid.org,El Centro dispone de un edificio principal co...
2,2,Reserva de la Biosfera de la Sierra del Rincón,Hayedo de Montejo,Desde Montejo de la Sierra por la M- 137 hacia...,Teléfono: 91 869 70 58 / 91 869 72 17,Email: sierradelrincon@fida.es,El “Hayedo de Montejo” constituye uno de los ...
3,3,Reserva de la Biosfera de la Sierra del Rincón,Senda del Agua y Laguna del Salmoral,28191 Prádena del Rincón (Madrid),Teléfono: 91 869 70 58 / 91 869 72 17,Email: sierradelrincon@fida.es,La “Senda del Agua” une el municipio de Práde...


In [3]:
p1

,Unnamed: 0,Centro,Acceso,Recepción,Aseo adaptado,Inodoro,Lavabo y accesorios del aseo,Senderos,Mobiliario y zonas de descanso,Salas de exposiciones,Sala de audiovisuales,Sala de talleres infantiles,Biblioteca,Itinerario de la Senda,Seguridad,Observatorio
0,0,Centro de Educación Ambiental Puente del Perdón,Para acceder desde el aparcamiento hasta el ed...,Se accede desde las áreas temáticas al aire li...,"Hay una cabina de aseo adaptada, independiente...",El inodoro tiene un espacio libre de acercamie...,El lavabo permite la aproximación frontal de u...,Las sendas que recorren el “Área de Recursos A...,Dispone de una zona de descanso al aire libre ...,NP,NP,NP,NP,NP,NP,NP
1,1,Centro de Educación Ambiental Manzanares,Desde el aparcamiento hasta el edificio princi...,El itinerario desde la puerta de acceso hasta ...,Hay una cabina de aseo adaptada para ambos sex...,El inodoro tiene un espacio libre de acercamie...,El lavabo permite la aproximación frontal de u...,NP,NP,La sala de exposición está situada en la plant...,Se accede directamente desde la recepción sin ...,La puerta de entrada tiene dos hojas de madera...,Está situada en la primera planta del edificio...,NP,NP,NP
2,2,Hayedo de Montejo,NP,NP,La “Senda del Río” no tiene aseos.,NP,NP,NP,No dispone de mobiliario o zona de descanso co...,NP,NP,NP,NP,"Tras algo más de 1 km de recorrido, con un pav...",La senda está delimitada de manera puntual por...,NP
3,3,Senda del Agua y Laguna del Salmoral,NP,NP,La “Senda del agua” no tiene aseos.,NP,NP,NP,Dispone de alguna zona de descanso accesible p...,NP,NP,NP,NP,NP,La laguna está delimitada por bordillos latera...,"El observatorio es una caseta de madera, donde..."
